In [1]:
from pathlib import Path
import numpy as np
import pandas as pd

In [2]:
dataset = "HelocDataset"
method = "GLOBE_CE"
model_name = "MultilayerPerceptron"
# model_name = "MultinomialLogisticRegression"

root = f"../models/{dataset}/{method}"

In [3]:
metrics = []
time_files = []

In [4]:
for i in range(5):
    path = Path(root) / f"fold_{i}" / f"cf_metrics_{model_name}.csv"
    df = pd.read_csv(path)
    metrics.append(df)

    path = Path(root) / f"fold_{i}" / "search_time.txt"
    with open(path, "r") as f:
        search_time = f.read()

    time_files.append(float(search_time))

In [5]:
merged_df = pd.concat(metrics, axis=0, ignore_index=True)

In [6]:
mean_ = merged_df.mean(axis=0)
std_ = merged_df.std(axis=0)

/Users/lukasz/miniconda3/envs/globe-ce/lib/python3.11/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


In [7]:
def format_mean_std(m, s):
    return f"{m:.2f} ± {s:.2f}"


# Create the merged table
formatted = pd.DataFrame(
    {col: [format_mean_std(mean_[col], std_[col])] for col in merged_df.columns}
).T

In [8]:
used_metrics = [
    "coverage",
    "validity",
    "proximity_continuous_euclidean",
    "prob_plausibility",
    "log_density_cf",
    "isolation_forest_scores_cf",
    "lof_scores_cf",
]

In [9]:
transposed = formatted.T
transposed = transposed[used_metrics]

In [10]:
transposed["search_time"] = [f"{np.mean(time_files):.2f} ± {np.std(time_files):.2f}"]

In [11]:
transposed

,coverage,validity,proximity_continuous_euclidean,prob_plausibility,log_density_cf,isolation_forest_scores_cf,lof_scores_cf,search_time
0,1.00 ± 0.00,0.99 ± 0.02,0.74 ± 0.18,0.12 ± 0.04,-inf ± nan,0.05 ± 0.01,2.22 ± 0.37,4.75 ± 0.22


In [12]:
"| " + " | ".join(transposed.loc[0].to_list()) + " |"

'| 1.00 ± 0.00 | 0.99 ± 0.02 | 0.74 ± 0.18 | 0.12 ± 0.04 | -inf ± nan | 0.05 ± 0.01 | 2.22 ± 0.37 | 4.75 ± 0.22 |'

In [13]:
t = [
    18.37492847442627,
    18.803892135620117,
    18.664883852005005,
    18.604572057724,
    18.54537868499756,
]